In [1]:
import pandas as pd
import numpy as np
import os
import parameters
from tqdm import tqdm


In [ ]:
parameters.KNN

In [ ]:
header = ["chr", "start", "end", "name", "score", "strand", "signalValue", "pValue", "qValue", "peak"]
all_chroms = {f"chr{i}" for i in range(1, 23)}
B = {"chr14", "chr19"}
C = {"chr1"}
A = all_chroms - B - C

data_train = {}
sorted_indexes = {}

n_sorted_indexes = 100


for cell_type in ["X1", "X2"]:
    train_df = pd.read_csv(f"Data/CAGE-train/{cell_type}_train_info.tsv", sep="\t") #, index_col="gene_name")

    data_train[cell_type] = np.zeros((len(train_df), parameters.N_BEDS, parameters.KNN, parameters.N_FEATURES_BED + 1))
    sorted_indexes[cell_type] = -np.ones((len(train_df), parameters.N_BEDS, n_sorted_indexes), dtype=int)
    
    for signal_index, signal in enumerate(os.listdir("Data/bed/")):
        bed = pd.read_csv(f"Data/bed/{signal}/{cell_type}.bed", names=header, sep="\t")
        bed["center"] = (bed["start"] + bed["end"]) // 2

        # joined_df = train_df.merge(bed, on="chr", how="inner", suffixes=('_train', '_bed'))
        # joined_df["distance"] = np.abs(joined_df["center"] - joined_df["TSS_start"])
        # joined_df = joined_df.groupby("gene_name").apply(lambda x: x.nsmallest(parameters.KNN, "distance"))
        # print(joined_df.head())

        for row_index, row in tqdm(train_df.iterrows(), total=len(train_df)):
            chr = row["chr"]
            TSS_start = row["TSS_start"]
            TSS_end = row["TSS_end"]
            same_chromosome = bed[bed["chr"] == chr].copy()
            same_chromosome["distance"] = np.abs(same_chromosome["center"] - TSS_start)
            closest_indexes = np.argsort(same_chromosome["distance"])[:n_sorted_indexes]
            knn = same_chromosome.iloc[closest_indexes[:parameters.KNN]]

            data_train[cell_type][row_index, signal_index, :, 0] = knn["signalValue"].values
            data_train[cell_type][row_index, signal_index, :, 1] = knn["distance"].values
            sorted_indexes[cell_type][row_index, signal_index, :len(closest_indexes)] = closest_indexes

        


 17%|█▋        | 2492/14310 [00:16<01:19, 147.91it/s]


KeyboardInterrupt: 

In [ ]:
X1["center"] = (X1["start"] + X1["end"]) / 2
X1[X1["signal"] == "DNase-bed"].head()

,chr,start,end,name,score,strand,signalValue,pValue,qValue,peak,signal,center
81541,chr1,181405,181555,.,0,.,12.379543,-1.0,-1.0,75,DNase-bed,181480.0
81542,chr1,267990,268140,.,0,.,12.379543,-1.0,-1.0,75,DNase-bed,268065.0
81543,chr1,629160,629310,.,0,.,22.263896,-1.0,-1.0,75,DNase-bed,629235.0
81544,chr1,629494,629644,.,0,.,10.732151,-1.0,-1.0,75,DNase-bed,629569.0
81545,chr1,629870,630020,.,0,.,3603.694491,-1.0,-1.0,75,DNase-bed,629945.0


In [ ]:
train_data = np.load("Data/train_data.npz", allow_pickle=True)

In [ ]:
X = train_data["x"]
X[:, :, 0].mean()
len(X)

32568